In [1]:
library(data.table)
require(caret)
library(dummies)
library(pROC)
DT = load("projetdatascience.rda")

Loading required package: caret
Loading required package: lattice
Loading required package: ggplot2
Warning message:
"package 'dummies' was built under R version 3.3.2"dummies-1.5.6 provided by Decision Patterns

Warning message:
"package 'pROC' was built under R version 3.3.2"Type 'citation("pROC")' for a citation.

Attaching package: 'pROC'

The following objects are masked from 'package:stats':

    cov, smooth, var



In [2]:
names(projetTrain)

[1] "Id"                  "Product_Info_1"      "Product_Info_2"     
  [4] "Product_Info_3"      "Product_Info_4"      "Product_Info_5"     
  [7] "Product_Info_6"      "Product_Info_7"      "Ins_Age"            
 [10] "Ht"                  "Wt"                  "BMI"                
 [13] "Employment_Info_1"   "Employment_Info_2"   "Employment_Info_3"  
 [16] "Employment_Info_4"   "Employment_Info_5"   "Employment_Info_6"  
 [19] "InsuredInfo_1"       "InsuredInfo_2"       "InsuredInfo_3"      
 [22] "InsuredInfo_4"       "InsuredInfo_5"       "InsuredInfo_6"      
 [25] "InsuredInfo_7"       "Insurance_History_1" "Insurance_History_2"
 [28] "Insurance_History_3" "Insurance_History_4" "Insurance_History_5"
 [31] "Insurance_History_7" "Insurance_History_8" "Insurance_History_9"
 [34] "Family_Hist_1"       "Family_Hist_2"       "Family_Hist_3"      
 [37] "Family_Hist_4"       "Family_Hist_5"       "Medical_History_1"  
 [40] "Medical_History_2"   "Medical_History_3"   "Medical_History_4"  
 [43] "Medical_History_5"   "Medical_History_6"   "Medical_History_7"  
 [46] "Medical_History_8"   "Medical_History_9"   "Medical_History_10" 
 [49] "Medical_History_11"  "Medical_History_12"  "Medical_History_13" 
 [52] "Medical_History_14"  "Medical_History_15"  "Medical_History_16" 
 [55] "Medical_History_17"  "Medical_History_18"  "Medical_History_19" 
 [58] "Medical_History_20"  "Medical_History_21"  "Medical_History_22" 
 [61] "Medical_History_23"  "Medical_History_24"  "Medical_History_25" 
 [64] "Medical_History_26"  "Medical_History_27"  "Medical_History_28" 
 [67] "Medical_History_29"  "Medical_History_30"  "Medical_History_31" 
 [70] "Medical_History_32"  "Medical_History_33"  "Medical_History_34" 
 [73] "Medical_History_35"  "Medical_History_36"  "Medical_History_37" 
 [76] "Medical_History_38"  "Medical_History_39"  "Medical_History_40" 
 [79] "Medical_History_41"  "Medical_Keyword_1"   "Medical_Keyword_2"  
 [82] "Medical_Keyword_3"   "Medical_Keyword_4"   "Medical_Keyword_5"  
 [85] "Medical_Keyword_6"   "Medical_Keyword_7"   "Medical_Keyword_8"  
 [88] "Medical_Keyword_9"   "Medical_Keyword_10"  "Medical_Keyword_11" 
 [91] "Medical_Keyword_12"  "Medical_Keyword_13"  "Medical_Keyword_14" 
 [94] "Medical_Keyword_15"  "Medical_Keyword_16"  "Medical_Keyword_17" 
 [97] "Medical_Keyword_18"  "Medical_Keyword_19"  "Medical_Keyword_20" 
[100] "Medical_Keyword_21"  "Medical_Keyword_22"  "Medical_Keyword_23" 
[103] "Medical_Keyword_24"  "Medical_Keyword_25"  "Medical_Keyword_26" 
[106] "Medical_Keyword_27"  "Medical_Keyword_28"  "Medical_Keyword_29" 
[109] "Medical_Keyword_30"  "Medical_Keyword_31"  "Medical_Keyword_32" 
[112] "Medical_Keyword_33"  "Medical_Keyword_34"  "Medical_Keyword_35" 
[115] "Medical_Keyword_36"  "Medical_Keyword_37"  "Medical_Keyword_38" 
[118] "Medical_Keyword_39"  "Medical_Keyword_40"  "Medical_Keyword_41" 
[121] "Medical_Keyword_42"  "Medical_Keyword_43"  "Medical_Keyword_44" 
[124] "Medical_Keyword_45"  "Medical_Keyword_46"  "Medical_Keyword_47" 
[127] "Medical_Keyword_48"  "target"

In [4]:
na.roughfix <- function(object, ...)
  UseMethod("na.roughfix")

na.roughfix.data.frame <- function(object, ...) {
  isfac <- sapply(object, is.factor)
  isnum <- sapply(object, is.numeric)
  if (any(!(isfac | isnum)))
    stop("na.roughfix only works for numeric or factor")
  roughfix <- function(x) {
    if (any(is.na(x))) {
      if (is.factor(x)) {
        freq <- table(x)
        x[is.na(x)] <- names(freq)[which.max(freq)]
      } else {
        x[is.na(x)] <- median(x, na.rm=TRUE)
      }
    }
    x
  }
  object[] <- lapply(object, roughfix)
  object
}

na.roughfix.default <- function(object, ...) {
  if (!is.atomic(object))
    return(object)
  d <- dim(object)
  if (length(d) > 2)
    stop("can't handle objects with more than two dimensions")
  if (all(!is.na(object)))
    return(object)
  if (!is.numeric(object))
    stop("roughfix can only deal with numeric data.")
  if (length(d) == 2) {
    hasNA <- which(apply(object, 2, function(x) any(is.na(x))))
    for (j in hasNA)
      object[is.na(object[, j]), j] <- median(object[, j], na.rm=TRUE)
  } else {
    object[is.na(object)] <- median(object, na.rm=TRUE)
  }
  object
}


In [3]:
str(train)

In [41]:
?tuneRF

Classes 'data.table' and 'data.frame':	51381 obs. of  128 variables:
 $ Id                 : int  70890 21013 29464 45241 71807 15909 71030 74756 52065 49609 ...
 $ Product_Info_1     : int  1 1 1 1 1 1 1 1 1 1 ...
 $ Product_Info_2     : chr  "A8" "A2" "D3" "A8" ...
 $ Product_Info_3     : int  26 26 10 26 26 10 26 10 26 26 ...
 $ Product_Info_4     : num  0.1463 0.0769 0.2308 0.0256 0.2308 ...
 $ Product_Info_5     : int  2 2 2 2 2 2 2 2 2 2 ...
 $ Product_Info_6     : int  3 3 3 3 3 3 3 3 1 1 ...
 $ Product_Info_7     : int  1 1 1 1 1 1 1 3 1 1 ...
 $ Ins_Age            : num  0.701 0.687 0.612 0.284 0.522 ...
 $ Ht                 : num  0.691 0.8 0.727 0.709 0.618 ...
 $ Wt                 : num  0.266 0.32 0.255 0.278 0.205 ...
 $ BMI                : num  0.444 0.426 0.391 0.447 0.405 ...
 $ Employment_Info_1  : num  0 0 0.06 0.031 0.18 0.024 0.046 0.04 0.105 0.118 ...
 $ Employment_Info_2  : int  1 1 9 9 14 9 9 9 9 9 ...
 $ Employment_Info_3  : int  3 3 1 1 1 1 1 1 1 1 ...
 $ E

In [3]:
#GETTING THE RATIO OF NA VALUE FOR EACH FEATURE and number of factors 
vec_names = names(projetTrain)
nb_lignes = 51381
vec_taux  = rep(0,length(vec_names))
vec_factors = rep(0,length(vec_names))
bene <- data.table(feature=vec_names, 
                   taux=vec_taux,
                   factors=vec_factors)
trainDF = as.data.frame(projetTrain)
index = 1

for(name in bene$feature){
  
  bene[index,taux := (sum(is.na( trainDF[name] ) )  / nb_lignes)*100] 
  bene[index,factors:=length(table(trainDF[name]))]
  index=index+1
}

In [7]:
#GETTING COLUMNS NAME THAT HAVE NA RATIO > 17 and COLUMNS THAT CAN BE FACTORISED
columnsToFactor = bene[factors<10]$feature
columnsToDrop  = bene[taux>17]$feature
print(bene[taux>0 ])

                feature       taux factors
 1:   Employment_Info_1  0.0350324    1767
 2:   Employment_Info_4 11.4555964     787
 3:   Employment_Info_6 18.3336253     909
 4: Insurance_History_5 42.6324906    2033
 5:       Family_Hist_2 48.2376754      68
 6:       Family_Hist_3 57.7372959      89
 7:       Family_Hist_4 32.2551138      68
 8:       Family_Hist_5 70.5105000      89
 9:   Medical_History_1 15.0133318     167
10:  Medical_History_10 99.0638563      93
11:  Medical_History_15 75.0588739     241
12:  Medical_History_24 93.6104786     219
13:  Medical_History_32 98.1549600      84


In [8]:
#GETTING RID OF COLUMNS THAT HAVE HIGH RATIO >42 OF NA VALUES
projetTrainClean = projetTrain[,(columnsToDrop):=NULL]
projetValidClean = projetValid[,(columnsToDrop):=NULL]

In [9]:
nrow(projetTrainClean[complete.cases(projetTrainClean)]) / nrow(projetTrainClean)

[1] 0.7517954

In [10]:
for (feature in columnsToFactor) {    
    if(feature != "target"){
            projetValidClean[,feature:=as.factor(get(feature)),with=FALSE]
            projetTrainClean[,feature:=as.factor(get(feature)),with=FALSE]

    }
}
projetTrainClean$Product_Info_2 = as.factor(projetTrainClean$Product_Info_2)
projetValidClean$Product_Info_2 = as.factor(projetValidClean$Product_Info_2)

In [11]:
projetValidClean$target = NA
pool = rbind(projetTrainClean,projetValidClean)
pool = na.roughfix.data.frame(pool)

train = as.data.table(pool)[0:nrow(projetTrainClean)]
valid = as.data.table(pool)[(nrow(projetTrainClean)+1):nrow(pool)]

In [12]:
#pool.dummy = dummy.data.frame(pool)
#train.dummy = as.data.table(pool.dummy)[0:nrow(projetTrainClean)]
#valid.dummy = as.data.table(pool.dummy)[(nrow(projetTrainClean)+1):nrow(pool.dummy)]

In [13]:
#valid.dummy[,target := NULL]

,Id,Product_Info_11,Product_Info_12,Product_Info_2A1,Product_Info_2A2,Product_Info_2A3,Product_Info_2A4,Product_Info_2A5,Product_Info_2A6,Product_Info_2A7,...,Medical_Keyword_440,Medical_Keyword_441,Medical_Keyword_450,Medical_Keyword_451,Medical_Keyword_460,Medical_Keyword_461,Medical_Keyword_470,Medical_Keyword_471,Medical_Keyword_480,Medical_Keyword_481
1,70893,1,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
2,20997,1,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,0,1
3,29427,1,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
4,45161,1,0,1,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
5,71802,1,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
6,15895,1,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
7,71014,1,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
8,74730,1,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
9,52094,1,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
10,49602,0,1,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0


# Prediction arbre simple

In [13]:
set.seed(00)
split = sample(nrow(projetTrainClean), floor(0.7*nrow(projetTrainClean)))
sub.train = train[split,]
sub.test  = train[-split,]


In [14]:
Y = sub.test$target
sub.test[,target:=NULL]

,Id,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,...,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48
1,6,1,E1,26,0.076923077,2,3,1,0.029850746,0.745454545,...,0,0,0,0,0,0,0,0,0,0
2,7,1,D4,10,0.487179487,2,3,1,0.164179104,0.672727273,...,0,0,0,0,0,0,0,0,0,0
3,15,1,D3,26,0.230769231,2,3,1,0.52238806,0.618181818,...,0,0,0,0,0,0,0,0,0,0
4,20,2,D1,26,0.487179487,2,3,1,0.223880597,0.781818182,...,0,0,0,0,0,0,0,0,0,0
5,23,1,A7,26,0,2,3,1,0.626865672,0.672727273,...,0,0,0,0,0,0,0,0,0,0
6,24,2,D4,26,0.487179487,2,3,1,0.208955224,0.745454545,...,0,1,0,0,0,0,0,0,0,0
7,32,1,D4,26,0.230769231,2,3,1,0.179104478,0.8,...,0,0,0,0,0,0,0,0,0,0
8,35,1,A6,26,0.230769231,2,3,1,0.268656716,0.781818182,...,0,0,0,0,0,0,0,0,0,0
9,39,1,D3,26,0.230769231,2,3,1,0.134328358,0.763636364,...,0,0,0,0,0,0,0,0,0,0
10,42,1,D3,26,0.487179487,2,1,1,0.28358209,0.709090909,...,0,0,0,0,0,0,0,0,0,0


In [15]:
library(rpart)
library(rpart.plot)

Warning message:
"package 'rpart' was built under R version 3.3.2"Warning message:
"package 'rpart.plot' was built under R version 3.3.2"

In [18]:
install.packages("randomForest",repos = 'http://cran.us.r-project.org')

Installing package into 'C:/Users/anirb/Documents/R/win-library/3.3'
(as 'lib' is unspecified)


package 'randomForest' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'randomForest'"


The downloaded binary packages are in
	C:\Users\anirb\AppData\Local\Temp\Rtmp46uYDW\downloaded_packages


In [16]:
target.tree = rpart(target~.,data=sub.train,control=rpart.control(minsplit=5,cp=0))

In [19]:
target.tree.optimal = prune(target.tree,cp=target.tree$cptable[which.min(target.tree$cptable[,4]),1])

In [27]:
Y.predicted = predict(target.tree.optimal , sub.test)
Y.predicted = as.data.table(Y.predicted)

In [28]:
Y.response = Y.predicted[,Y.predicted:=ifelse(Y.predicted>0.5,1,0)]

In [38]:
table(Y,Y.response$Y.predicted)
# 0.81
# pretty neat for a simple rpart

   
Y      0    1
  0 8661 1597
  1 1226 3931